# labelstud

In [84]:
# !pip install label-studio-sdk
from label_studio_sdk import Client

In [83]:
LABEL_STUDIO_URL = 'http://3.67.8.157:8080/'
API_KEY = '11da51f59ebf49a4e93af5a491734184a52178a7'

In [85]:
ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
print(ls.check_connection())

{'status': 'UP'}


In [86]:
for project in ls.get_projects():
    if project.title == 'Classification':
        break
project.title

'Classification'

In [92]:
# tasks = project.get_tasks(selected_ids=[i for i in range(50, 80)])
# tasks[0]['data']
labeled_tasks = project.get_labeled_tasks()

In [98]:
labeled_tasks[0]['data']

{'text': 'косметика ухаживающая',
 'is_phrase': 'multiword',
 'frequency': 98414,
 'link': 'https://google.com/search?q=косметика+ухаживающая'}

In [152]:
def get_annotation(labeled_tasks, id):
    return labeled_tasks[id]['annotations'][0]['result'][0]['value']['choices'][0]

In [158]:
get_annotation(labeled_tasks, 0)

'Упаковка'

### Work with project

In [149]:
ls.get_projects()[0].title

'Test labeling config'

___

# Init model

In [1]:
from gensim.models.fasttext import FastText, load_facebook_model

In [2]:
pip show gensim

Name: gensim
Version: 4.2.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: c:\users\bimas\anaconda3\lib\site-packages
Requires: Cython, scipy, numpy, smart-open
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
model = load_facebook_model('cc.ru.300.bin')

In [2]:
model2 = load_facebook_model('model.bin')


KeyboardInterrupt



In [28]:
res2 = model.wv.evaluate_word_analogies(analogies='ru_analogy.txt')
res2

(0.6066408143480368,
 [{'section': 'capital-common-countries',
   'correct': [('АФИНЫ', 'ГРЕЦИЯ', 'БАГДАД', 'ИРАК'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'БАНГКОК', 'ТАИЛАНД'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'ПЕКИН', 'КИТАЙ'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'БЕРЛИН', 'ГЕРМАНИЯ'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'БЕРН', 'ШВЕЙЦАРИЯ'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'КАИР', 'ЕГИПЕТ'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'КАНБЕРРА', 'АВСТРАЛИЯ'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'ХАНОЙ', 'ВЬЕТНАМ'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'ГАВАНА', 'КУБА'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'ХЕЛЬСИНКИ', 'ФИНЛЯНДИЯ'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'ИСЛАМАБАД', 'ПАКИСТАН'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'КАБУЛ', 'АФГАНИСТАН'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'МАДРИД', 'ИСПАНИЯ'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'МОСКВА', 'РОССИЯ'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'ОСЛО', 'НОРВЕГИЯ'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'ОТТАВА', 'КАНАДА'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'ПАРИЖ', 'ФРАНЦИЯ'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'РИМ', 'ИТАЛИЯ'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'СТОКГОЛЬМ', 'ШВЕЦИЯ'),
    ('АФИНЫ', 'ГРЕЦИЯ', 'ТЕГЕРАН', 'ИРАН'),
    ('АФИН

## Оценка модели

In [4]:
import numpy as np

def similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)

    if n1 < 1e-6 or n2 < 1e-6:
        return 0.0
    else:
        return np.dot(v1, v2) / n1 / n2

def word_similarity(model, w1, w2):
    return similarity(model.get_vector(w1), model.get_vector(w2))

def sentence_similarity(model, t1, t2):
    return similarity(model.get_sentence_vector(t1), model.get_sentence_vector(t2))

In [5]:
word_similarity(model.wv, "Привет", "Приветик")

0.76207703

In [6]:
sentence_similarity(model.wv, "Привет", "Приветствую")

0.96948314

In [7]:
model.wv.most_similar('Привет')

[('привет', 0.7965796589851379),
 ('Приветик', 0.7620770931243896),
 ('.Привет', 0.7517420053482056),
 ('Приветики', 0.7352913022041321),
 ('ПРивет', 0.726387083530426),
 ('Привееет', 0.7079933285713196),
 ('Здраствуй', 0.70646071434021),
 ('Привет-привет', 0.6927946209907532),
 ('Здрасте', 0.6923279166221619),
 ('Здарова', 0.6814324855804443)]

In [8]:
model.wv.most_similar_cosmul('Привет')

[('привет', 0.8982889652252197),
 ('Приветик', 0.881037712097168),
 ('.Привет', 0.8758701682090759),
 ('Приветики', 0.8676447868347168),
 ('ПРивет', 0.8631926774978638),
 ('Привееет', 0.8539958000183105),
 ('Здраствуй', 0.8532295227050781),
 ('Привет-привет', 0.8463964462280273),
 ('Здрасте', 0.8461631536483765),
 ('Здарова', 0.8407154679298401)]

# Дообучение

In [7]:
import pandas as pd

In [8]:
data = pd.read_csv('proc.zip')
data.head()

,text_review,review_id,preprocessed_review,lemmas
0,Мне если честно аромат не очень.... Всегда ста...,75b8359b-2336-4759-880d-45c531c3470c,честно аромат очень стараться прислушиваться а...,я если честно аромат не очень всегда стараться...
1,Туалетная вода от «Nina Ricci» - Nina L’Eau.Пр...,e7077664-72ae-49f5-a1b8-7b683fd07728,туалетный вода nina ricci nina приобрести инте...,туалетный вода от nina ricci nina l eau приобр...
2,Очень приятный и свежий аромат. Ярко стартует ...,f99d2450-90dc-4914-9c02-6cf1d41de700,очень приятный свежий аромат ярко стартовать ц...,очень приятный и свежий аромат ярко стартовать...
3,Cтал счастливым обладателем запечатанной миниа...,06fc790e-6dde-4835-ac00-5c1263884033,cталый счастливый обладатель запечатать миниат...,cталый счастливый обладатель запечатать миниат...
4,"Увидев на стенде Летуаль новинку от Shiseido, ...",4abc59a2-ebf0-44ed-b9fe-0ebe71f57807,увидеть стенд летуаля новинка shiseido смочь п...,увидеть на стенд летуаля новинка от shiseido н...


In [9]:
data.dropna(inplace=True)

In [6]:
model.build_vocab(corpus_iterable=data['lemmas'][:10], update=True)

In [14]:
import pymorphy2
from functools import lru_cache
from multiprocessing import Pool
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
import re

# pymorphy2 - библиотека методов для морфологического анализа (в том числе лемматизации) русскоязычного текста
m = pymorphy2.MorphAnalyzer()

# убираем все небуквенные символы
regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text)
    except:
        return []

In [9]:
def lemmatize(text, pymorphy=m):
    try:
        return " ".join([pymorphy.parse(w)[0].normal_form for w in text])
    except:
        return " "    

In [10]:
def clean_text(text):
    return lemmatize(words_only(text))

In [ ]:
with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, data['text_review']), total=len(data)))
    
data['lemmas'] = lemmas
data.head()

C:\Users\bimas\AppData\Local\Temp/ipykernel_11956/4155227129.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  lemmas = list(tqdm(p.imap(clean_text, data['text_review']), total=len(data)))


  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
if __name__ == '__main__':
    data = pd.read_csv('out.zip')
    with Pool(8) as p:
        lemmas = list(tqdm(p.imap(clean_text, data['text_review'][:10]), total=len(data[:10])))

C:\Users\bimas\AppData\Local\Temp/ipykernel_15872/3570654016.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  lemmas = list(tqdm(p.imap(clean_text, data['text_review'][:10]), total=len(data[:10])))


  0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
import pandas as pd
data = pd.read_csv('proc.zip')
data.head()

,text_review,review_id,preprocessed_review,lemmas
0,Мне если честно аромат не очень.... Всегда ста...,75b8359b-2336-4759-880d-45c531c3470c,честно аромат очень стараться прислушиваться а...,я если честно аромат не очень всегда стараться...
1,Туалетная вода от «Nina Ricci» - Nina L’Eau.Пр...,e7077664-72ae-49f5-a1b8-7b683fd07728,туалетный вода nina ricci nina приобрести инте...,туалетный вода от nina ricci nina l eau приобр...
2,Очень приятный и свежий аромат. Ярко стартует ...,f99d2450-90dc-4914-9c02-6cf1d41de700,очень приятный свежий аромат ярко стартовать ц...,очень приятный и свежий аромат ярко стартовать...
3,Cтал счастливым обладателем запечатанной миниа...,06fc790e-6dde-4835-ac00-5c1263884033,cталый счастливый обладатель запечатать миниат...,cталый счастливый обладатель запечатать миниат...
4,"Увидев на стенде Летуаль новинку от Shiseido, ...",4abc59a2-ebf0-44ed-b9fe-0ebe71f57807,увидеть стенд летуаля новинка shiseido смочь п...,увидеть на стенд летуаля новинка от shiseido н...


In [11]:
%%time
model.train(corpus_iterable=data['lemmas'][:10].values, 
            epochs=1, 
            total_examples=model.corpus_count, 
            total_words=model.corpus_total_words)

Wall time: 41.4 s


(253, 12853)

In [75]:
data.dropna(inplace=True)

In [10]:
sentence_corpus = [line.split(' ') for line in data['lemmas'].values if len(line) > 10 and len(line) <= 300]

In [25]:
# model.build_vocab(data['lemmas'][:10].values, update=True)

In [24]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(
    min_count=10,
    window=2,
#     size=300,
    negative=10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1)

In [25]:
w2v_model.build_vocab(sentence_corpus)

In [26]:
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1, compute_loss=True)

(295, 1350)

In [27]:
w2v_model.get_latest_training_loss()

4213.04150390625

In [19]:
model.vector_size

300

In [15]:
model.build_vocab(sentence_corpus)

In [ ]:
102825040945

model.corpus_total_words

In [19]:
%%time
model.train(corpus_iterable=sentence_corpus, 
            epochs=4, 
            total_examples=model.corpus_count, 
            total_words=model.corpus_total_words,
            compute_loss=True)

Wall time: 29.6 s


(335427, 1640824)

In [21]:
model.get_latest_training_loss()

0.0

In [43]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import word2vec

In [56]:
import copy

In [82]:
import time

In [67]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

from pylab import rcParams

%matplotlib inline
%config InlineBackend.figure_format = 'png'
rcParams['figure.figsize'] = 10, 5

sns.set(style='darkgrid')
sns.set(font_scale=1.5)

In [63]:
class callback(CallbackAny2Vec):
    '''Callback for Word2vec with resetting loss on the end of each epoch.'''

    def __init__(self):
        self.epoch = 1

        self.epoch = 1
        self.losses = []
        self.cumu_loss = 0.0
        self.previous_epoch_time = time.time()

        self.best_model = None
        self.best_loss = 1e+30

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()

        norms = [np.linalg.norm(v) for v in model.wv.vectors]
        now = time.time()
        epoch_seconds = now - self.previous_epoch_time
        self.previous_epoch_time = now
        self.cumu_loss += float(loss)
        print(f"Loss after epoch {self.epoch}: {loss} (cumulative loss so far: {self.cumu_loss}) "+\
              f"-> epoch took {round(epoch_seconds, 2)} s - vector norms min/avg/max: "+\
              f"{round(float(min(norms)), 2)}, {round(float(sum(norms)/len(norms)), 2)}, {round(float(max(norms)), 2)}")
        self.epoch += 1

        self.losses.append(float(loss))

        # reset loss inside model
        model.running_training_loss = 0.0

        if loss < self.best_loss:
            self.best_model = copy.deepcopy(model)
            self.best_loss = loss

        if self.epoch % 50 == 0:
            self.plot(path="w2v_training_loss.png")

    def plot(self, path):
        fig, (ax1) = plt.subplots(ncols=1, figsize=(6, 6))
        ax1.plot(self.losses, label="loss per epoch")
        plt.legend()
        plt.savefig(path)
        plt.close()
        print("Plotted loss.")

In [90]:
model = word2vec.Word2Vec(
        vector_size=300,
        min_count=1,
        window=5,
        workers=4,
        sg=1, 
        seed=46,
#         iter=200,
        compute_loss=True)

In [91]:
print("building vocabulary...")
# sentence corpus is like : [["this", "is", "a", "dog"], ["he", "is", "a", "student"], ... ]
model.build_vocab(sentence_corpus)

print("training Word2Vec...")
callbacker = callback()
model.train(
        sentence_corpus,
#         epochs=model.iter,
        epochs=100,
        total_examples=model.corpus_count,
        compute_loss=True,
        callbacks=[callbacker],
    )

building vocabulary...
training Word2Vec...
Loss after epoch 1: 1410129.25 (cumulative loss so far: 1410129.25) -> epoch took 0.69 s - vector norms min/avg/max: 0.03, 0.58, 4.08
Loss after epoch 2: 1194407.625 (cumulative loss so far: 2604536.875) -> epoch took 0.73 s - vector norms min/avg/max: 0.03, 0.87, 4.59
Loss after epoch 3: 1149691.0 (cumulative loss so far: 3754227.875) -> epoch took 0.72 s - vector norms min/avg/max: 0.03, 1.07, 4.59
Loss after epoch 4: 1121507.375 (cumulative loss so far: 4875735.25) -> epoch took 0.72 s - vector norms min/avg/max: 0.03, 1.22, 4.82
Loss after epoch 5: 1100877.75 (cumulative loss so far: 5976613.0) -> epoch took 0.73 s - vector norms min/avg/max: 0.03, 1.36, 5.02
Loss after epoch 6: 1077248.0 (cumulative loss so far: 7053861.0) -> epoch took 0.76 s - vector norms min/avg/max: 0.03, 1.49, 5.31
Loss after epoch 7: 1060386.25 (cumulative loss so far: 8114247.25) -> epoch took 0.73 s - vector norms min/avg/max: 0.03, 1.61, 5.47
Loss after epoch 8

(32740250, 41020600)

In [80]:
len(sentence_corpus[4801])

82

In [92]:
model.get_latest_training_loss()

0.0

In [97]:
model.score('2', '2')

RuntimeError: We have currently only implemented score for the hierarchical softmax scheme, so you need to have run word2vec with hs=1 and negative=0 for this to work.